In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

# Notes Summarizer App

In [7]:
from openai import OpenAI

def get_response(prompt):
    client = OpenAI()
    response = client.chat.completions.create(model="gpt-3.5-turbo-1106", 
                             messages=
                             [
                                 {"role": "system", "content": "You are a helpful research assistant."},
                                 {"role": "user", "content": prompt}   
                             ],
                             temperature=0.0,
                             n = 1
                             )
    return response.choices[0].message.content

notes_path = "./Reinventing-explanation.txt"
with open(notes_path, "r") as f:
    text = f.read()

get_response(f"Summarize this text: {text}")

'In his essay "Reinventing Explanation," Michael Nielsen discusses the potential of non-traditional media as a means to create deeper explanations of scientific ideas. He emphasizes the importance of understanding the potential of these media for serious explanations, rather than just popularization or education. Nielsen believes that non-traditional media can offer opportunities for explanations that are deeper than conventional verbal and symbolic explanations. He also explores the design of a tool for explaining scientific ideas using non-traditional media. Additionally, he touches on the concept of optimization and the goal of finding a set of weights that minimizes the loss function.'

Ok , great we can get a basic summary easy using just ChatGPT, but how about if we want to be able to specify different types of summaries? 
What if the contents we want to summarize are bigger than the context length of our model?

Let's address these issues by using langchain for summarization.

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import TextLoader
from IPython.display import Markdown, display

With langchain, summarization is as simple as this:

In [9]:
docs = TextLoader(notes_path).load()
llm = ChatOpenAI()
chain = load_summarize_chain(llm, chain_type="stuff") #stuff means everything goes inside the model's prompt
chain.run(docs)

"Michael Nielsen discusses the potential of non-traditional media for creating deeper explanations of scientific ideas, focusing on the design of media to explain scientific concepts. He emphasizes the importance of developing a vocabulary for digital explanation and using prototypes to make progress in this area. Nielsen also touches on the concept of Simpson's paradox and optimization in the context of understanding and manipulating mathematical objects through innovative mediums."

But as always, we would like to go beyond, why not set up a prompt template to allow for some customization of the summaries?

In [10]:
prompt_template = """Write a concise summary of the following text:

{text}

The summary should be in this style: {style}.
Let's think step by step.

SUMMARY:
"""

prompt_summarization = ChatPromptTemplate.from_template(prompt_template)
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt_summarization)
summary = chain.run({"input_documents": docs, "style": "bullet points"})
Markdown(summary)

- Michael Nielsen explores the use of non-traditional media as a way to provide deeper explanations of scientific ideas.
- He emphasizes the importance of understanding the potential of non-traditional media for serious explanations, rather than just for popularization or education.
- Nielsen focuses on the design of media to explain scientific ideas in a more comprehensive way.
- He mentions the lack of a basic vocabulary of digital explanation and suggests constructing prototypes to learn from them.
- Nielsen discusses the concept of Simpson's paradox and optimization in the context of explaining scientific ideas using non-traditional media.

Ok, let's wrap this into a streamlit app to allow for a nice looking user interface.

[Notes summarize example](https://platform.openai.com/examples/default-meeting-notes-summarizer)